In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from tqdm import *
import os
from joblib import Parallel, delayed
import pickle
from datetime import datetime
from dateutil.relativedelta import relativedelta

from rqdatac import *
from rqfactor import *
from rqfactor import Factor
from rqfactor.extension import *


init("13522652015", "123456")
import rqdatac

import seaborn as sns
import matplotlib.pyplot as plt

from read_stock_csv import read_stock_csv_files

# 关闭通知
import warnings

warnings.filterwarnings("ignore")
import logging

logging.getLogger().setLevel(logging.ERROR)

In [34]:
order_book_ids = [
    "000001.XSHE",
    "000002.XSHE",
    "000008.XSHE",
    "000009.XSHE",
    "000017.XSHE",
    "000066.XSHE",
    "000089.XSHE",
    "300024.XSHE",
    "300113.XSHE",
    "600000.XSHG",
    "600004.XSHG",
    "600007.XSHG",
    "600017.XSHG",
    "600028.XSHG",
    "600900.XSHG",
    "600905.XSHG",
    "600916.XSHG",
]
start_date = "2025-01-01"
end_date = "2025-07-01"
turnover_ratio_rq = get_turnover_rate(order_book_ids, start_date, end_date)
turnover_ratio_rq = turnover_ratio_rq.today.unstack("order_book_id")
turnover_ratio_rq.loc[start_date:end_date]

order_book_id,000001.XSHE,000002.XSHE,000008.XSHE,000009.XSHE,000017.XSHE,000066.XSHE,000089.XSHE,300024.XSHE,300113.XSHE,600000.XSHG,600004.XSHG,600007.XSHG,600017.XSHG,600028.XSHG,600900.XSHG,600905.XSHG,600916.XSHG
tradedate,,,,,,,,,,,,,,,,,
2025-01-02,0.9377,1.2171,4.0793,0.9371,7.9960,3.6115,0.7939,4.8673,8.2293,0.2688,1.1227,0.3897,1.5832,0.1813,0.4337,0.5500,1.2779
2025-01-03,0.5950,1.1577,4.3850,0.5631,19.5554,3.4957,0.6466,5.3889,6.8505,0.1727,0.8588,0.2686,1.1027,0.1710,0.3942,0.3280,1.3294
2025-01-06,0.5594,0.8756,3.3821,0.4234,11.2614,2.6218,0.4660,3.5673,4.8131,0.2719,0.5277,0.2323,1.0018,0.1365,0.3310,0.3559,0.9689
2025-01-07,0.3854,0.7415,2.6489,0.3977,12.5165,2.8235,0.4784,4.4641,4.2612,0.1372,0.5578,0.2642,0.8519,0.0976,0.2728,0.2196,0.5746
2025-01-08,0.5475,0.9393,3.2554,0.6985,10.9098,2.9480,0.4487,7.8823,5.0756,0.1643,0.5698,0.3029,1.0499,0.1679,0.3775,0.3306,0.9175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-25,0.8632,0.8448,1.8555,2.3709,6.9277,2.1946,0.3835,2.7409,3.4577,0.1919,0.4640,0.5114,0.9587,0.1564,0.3264,0.3425,1.1736
2025-06-26,1.1206,0.6799,1.7470,1.9132,4.4689,2.4072,0.4759,1.9435,3.2529,0.2260,0.4460,0.7174,0.8256,0.1224,0.2939,0.2308,0.8982
2025-06-27,1.2252,0.8097,1.1939,1.3493,4.3308,2.7996,0.3629,1.6516,2.4689,0.4944,0.4368,0.2664,0.8942,0.1342,0.6142,0.2792,0.8775


In [32]:
turnover_comparison

,000001.XSHE,000002.XSHE,000009.XSHE,000066.XSHE,000089.XSHE,600000.XSHG,600004.XSHG,600007.XSHG,600017.XSHG,600028.XSHG
tradedate,,,,,,,,,,
2025-01-02,0.9377,1.2171,0.9371,3.6172,0.7939,0.2688,1.1227,0.3897,1.5832,0.1813
2025-01-03,0.5950,1.1577,0.5631,3.5013,0.6466,0.1727,0.8588,0.2686,1.1027,0.1710
2025-01-06,0.5594,0.8756,0.4234,2.6260,0.4660,0.2719,0.5277,0.2323,1.0018,0.1365
2025-01-07,0.3854,0.7415,0.3977,2.8280,0.4784,0.1372,0.5578,0.2642,0.8519,0.0976
2025-01-08,0.5475,0.9393,0.6985,2.9527,0.4487,0.1643,0.5698,0.3029,1.0499,0.1679
...,...,...,...,...,...,...,...,...,...,...
2025-06-25,0.8632,0.8448,2.3709,2.2531,0.3835,0.1919,0.4640,0.5114,0.9587,0.1564
2025-06-26,1.1206,0.6799,1.9132,2.4714,0.4759,0.2260,0.4460,0.7174,0.8256,0.1224
2025-06-27,1.2252,0.8097,1.3493,2.8743,0.3629,0.4944,0.4368,0.2664,0.8942,0.1342


In [33]:
turnover_ratio_rq

order_book_id,000001.XSHE,000002.XSHE,000009.XSHE,000066.XSHE,000089.XSHE,600000.XSHG,600004.XSHG,600007.XSHG,600017.XSHG,600028.XSHG
tradedate,,,,,,,,,,
2025-01-02,0.9377,1.2171,0.9371,3.6115,0.7939,0.2688,1.1227,0.3897,1.5832,0.1813
2025-01-03,0.5950,1.1577,0.5631,3.4957,0.6466,0.1727,0.8588,0.2686,1.1027,0.1710
2025-01-06,0.5594,0.8756,0.4234,2.6218,0.4660,0.2719,0.5277,0.2323,1.0018,0.1365
2025-01-07,0.3854,0.7415,0.3977,2.8235,0.4784,0.1372,0.5578,0.2642,0.8519,0.0976
2025-01-08,0.5475,0.9393,0.6985,2.9480,0.4487,0.1643,0.5698,0.3029,1.0499,0.1679
...,...,...,...,...,...,...,...,...,...,...
2025-06-25,0.8632,0.8448,2.3709,2.1946,0.3835,0.1919,0.4640,0.5114,0.9587,0.1564
2025-06-26,1.1206,0.6799,1.9132,2.4072,0.4759,0.2260,0.4460,0.7174,0.8256,0.1224
2025-06-27,1.2252,0.8097,1.3493,2.7996,0.3629,0.4944,0.4368,0.2664,0.8942,0.1342
